In [2]:
import sys,os
import io
import re
import math
import requests
import zipfile
import pandas as pd
import numpy as np
import html
import arxiv
import time
import json
import time

import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

In [3]:

def query_and_save(id_list, items):
    
    #getting the abstract directly from the api
    papers = arxiv.query(id_list=id_list)
    #print("Queried {} files".format(len(papers)))
    #sleep to prevent spamming api
    time.sleep(1)
    
    for i,p in enumerate(papers):
        pid = id_list[i]
        item = items[pid]
        if not os.path.exists("abstracts/"+item["folder"]):
            os.mkdir("abstracts/"+item["folder"])
        with open("abstracts/"+item["folder"]+"/"+item["fn"]+".json", "w") as f:
            json.dump(p, f)

def iterate_files(limit=100):
    """ iterate our full text files, 
        only query items which don't yet exist
    """
    
    items = {}
    id_list = []
    
    count=0
    queries = 0
    max_queries = 1000000

    for root,d_names,f_names in os.walk("fulltext"):
        #print (root, d_names, f_names)
        for fname in f_names:
            lst = fname.split(".")
            folder = "arxiv"
            fn = ""
            if len(lst) == 2:
                x = lst[0]
                idname = x[:-7]+"/"+x[-7:]
                folder = x[:-7]
                fn = x[-7:]
            else:
                idname = lst[0]+"."+lst[1]
                fn = idname
            
            if os.path.exists("abstracts/"+folder+"/"+fn+".json"):
                #print("skipping: {}".format(fn))
                continue

            items[idname] = {
                "id":idname,
                "fulltext":root+"/"+fname,
                "folder":folder,
                "fn":fn,
            }
            id_list.append(idname)

            count+=1 
            
            if count > limit:
                query_and_save(id_list, items)
                queries += 1
                #reset variables count, id_list, items, here
                count = 0
                id_list = []
                items = {}
            
                if queries % 1000 == 0:
                    print("Query: {}".format(queries))
            
            if queries >= max_queries:
                break
        
        if queries >= max_queries:
            break
            
        if count > limit:
            break
        
iterate_files()
print("Completed Download")


Completed Download


## Now connect the abstracts to the full text into one dataset

- Remove the abstracts from the fulltext 

In [3]:
count = 0
not_found = 0
found = 0
exp = 0
mis = []

for root,d_names,f_names in os.walk("fulltext"):
    #print (root, d_names, f_names)
    for fname in f_names:
        #print(fname)
        lst = fname.split(".")
        folder = "arxiv"
        fn = ""
        if len(lst) == 2:
            x = lst[0]
            idname = x[:-7]+"/"+x[-7:]
            folder = x[:-7]
            fn = x[-7:]
        else:
            idname = folder+"/"+lst[0]+"."+lst[1]
            fn = idname
        
        #read fulltext
        with open(root+"/"+fname) as f:
            fulltext = f.read()
        
        #get corresponding abstract
        try:
            with open("abstracts/"+idname+".json") as f:
                d = json.load(f)
            abstract = d["summary"]
        except Exception as e:
            mis.append("abstracts/"+idname+".json")
        
        try:
            m = re.search(abstract, fulltext)
            if m:
                #print("Abstract found")
                found +=1 
            else:
                #print("Abstract not found")
                not_found +=1 
        except Exception as e:
            #print(e)
            exp+=1
            
        count += 1
        

        
print("Found: {} Not Found: {} Exception: {}".format(found, not_found, exp))        


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Possible set difference at position 287
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Possible set difference at position 64
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Possible set difference at position 472
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Possible set difference at position 192
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Possible set difference at position 54
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Possible set difference at position 360
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Possible nested set at position 54
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: Future

FileNotFoundError: [Errno 2] No such file or directory: 'abstracts/q-bio/0606023.json'

In [4]:
print(found)
print(not_found)

3031
489273


In [2]:
print(3031/(3031+489273))

0.006156764925736943


## New Plan

Since Our "summaries" are not exact matches in the abstracts except for 0.6% of cases (negligble) we can assume that the summaries are in fact "summaries" and not abstracts. 

Masking or removing is essentially not required but can be included in future datasets

We have a large corpus of additional "gold standard" summaries

In [6]:
count = 0
not_found = 0

for root,d_names,f_names in os.walk("fulltext"):
    #print (root, d_names, f_names)
    for fname in f_names:
        #print(fname)
        lst = fname.split(".")
        folder = "arxiv"
        fn = ""
        if len(lst) == 2:
            x = lst[0]
            idname = x[:-7]+"/"+x[-7:]
            folder = x[:-7]
            fn = x[-7:]
        else:
            idname = folder+"/"+lst[0]+"."+lst[1]
            fn = lst[0]+"."+lst[1]
            
        dataset_fp = "dataset/"+folder+"."+fn+".json"
        
        if os.path.exists(dataset_fp):
            #skip overwriting
            count += 1
            continue
        
        #read fulltext
        with open(root+"/"+fname) as f:
            fulltext = f.read()
        
        #get corresponding abstract
        try:
            with open("abstracts/"+idname+".json") as f:
                d = json.load(f)
            abstract = d["summary"]
        except Exception as e:
            not_found += 1
        
        #combine fulltext into our new json blob
        d["fulltext"] = fulltext
        
        
        with open(dataset_fp, "w") as out:
            json.dump(d, out)
            
            count += 1
        

        
print("Found: {} Not Found: {} Exception: {}".format(count, not_found))      

NameError: name 'found' is not defined

In [7]:
print(count)

624221


In [4]:

current = 0
limit = 100

papers = []
for root,d_names,f_names in os.walk("dataset"):
    for fname in f_names:
        with open(root+"/"+fname,'r') as f:
            d = json.load(f)
        papers.append(d)
        current += 1
        if current > limit:
            break
    if current > limit:
        break


In [9]:
for k in papers[0]:
    print(k)

id
guidislink
updated
updated_parsed
published
published_parsed
title
title_detail
summary
summary_detail
authors
author_detail
author
links
arxiv_comment
arxiv_primary_category
tags
pdf_url
affiliation
arxiv_url
journal_reference
doi
fulltext


In [7]:
papers[0]["summary"]

'We study a finite range spin glass model in arbitrary dimension, where the\nintensity of the coupling between spins decays to zero over some distance\n$\\gamma^{-1}$. We prove that, under a positivity condition for the interaction\npotential, the infinite-volume free energy of the system converges to that of\nthe Sherrington-Kirkpatrick model, in the Kac limit $\\gamma\\to0$. We study the\nimplication of this convergence for the local order parameter, i.e., the local\noverlap distribution function and a family of susceptibilities to it\nassociated, and we show that locally the system behaves like its mean field\nanalogue. Similar results are obtained for models with $p$-spin interactions.\nFinally, we discuss a possible approach to the problem of the existence of long\nrange order for finite $\\gamma$, based on a large deviation functional for\noverlap profiles. This will be developed in future work.'

In [6]:
papers[0]["fulltext"][:1000]

'Finite-range spin glasses in the Kac limit: free energy and local\nobservables\nSilvio Franz [‡], Fabio Lucio Toninelli [†]\n\narXiv:cond-mat/0404132v1 [cond-mat.dis-nn] 6 Apr 2004\n\n[‡] The Abdus Salam International Center for Theoretical Physics, Condensed Matter Group\nStrada Costiera 11, P.O. Box 586, I-34100 Trieste, Italy\n[†] Institut für Mathematik, Universität Zürich\nWinterthurerstrasse 190, CH-8057 Zürich, Switzerland\n(Dated: November 2, 2018)\nWe study a finite range spin glass model in arbitrary dimension, where the intensity\nof the coupling between spins decays to zero over some distance γ −1 . We prove that,\nunder a positivity condition for the interaction potential, the infinite-volume free\nenergy of the system converges to that of the Sherrington-Kirkpatrick model, in the\nKac limit γ → 0. We study the implication of this convergence for the local order\nparameter, i.e., the local overlap distribution function and a family of susceptibilities\nto it associated, a